In [72]:
import pandas as pd
import numpy as np
import keras
import jieba
from re import sub

In [73]:
#读取数据
train_data = pd.read_csv('sohu.csv')
train_data

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
5,pic,10月4日十佳图：李娜小德球迷抢眼 大威显文静责编:马克杰 日期:2013-10-4 小德的...
6,pic,高清：小德速胜高举双臂庆祝 鼓掌向球迷们致意责编:马克杰 日期:2013-10-4 小德鼓掌...
7,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
8,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
9,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."


In [74]:
#数据预处理
train_data['label'].unique()

array(['pic', 'news', 'sports', 'business', 'caipiao', 'yule', 'mil', 'cul'], dtype=object)

In [75]:
train_data['label']=train_data['label'].map({'pic': 0, 'news': 1, 'sports' : 2, 'business' : 3, 'caipiao' : 4, 'yule' : 5, 'mil' : 6, 'cul' : 7})
train_data['label'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [76]:
for i in train_data['text']:
    #去除文本中的标点符号
    i = sub("[\s+\·<>?《》“”.\!【】\/_,$%^*(：\]\[\-:;+\\']+|[+——！，。？、~@#￥%……&*（）]+","",i)
    #分词
    i = jieba.cut(i, cut_all = True)
train_data['text'] = train_data['text'].apply(lambda x:' '.join(jieba.cut(x)))
train_data

,label,text
0,0,高清 ： 彭帅 谢淑薇 遗憾 遭 逆转 出局 击掌 互相 鼓励 责编 : 马克 杰 ...
1,1,河南 周口 路边 秸秆 燃烧 2 岁 幼儿 被 熏 晕 ( 图 ) 10 月 3...
2,1,消息 称 138 名 中国 劳工 在 菲律宾 被 抓 使馆 正 核实 【 使馆 正 核实...
3,1,越南 重要 军事 领导人 武元甲 大将 去世 享年 102 岁 据 美国 媒体 10 月...
4,1,西沙 搜救 已 发现 6 具 遇难 渔民 遗体 尚有 52 人 失踪 【 已 发现 6 ...
5,0,10 月 4 日 十佳 图 ： 李娜 小德 球迷 抢眼 大威 显 文静 责编 : 马克 ...
6,0,高清 ： 小德 速胜 高举 双臂 庆祝 鼓掌 向 球迷 们 致意 责编 : 马克 杰 ...
7,2,卡努特 伤愈 将 首发 战辽足 斯塔诺 ： 辽足 不好 对付 高清 : 国安 踩场 卡努...
8,1,美 “ 宙斯 盾 ” 系统 成功 进行 反导 试验 新华网 华盛顿 10 月 4 日电 ( ...
9,1,3 中国 建筑 工 俄罗斯 身亡 【 3 中国 建筑 工 俄罗斯 身亡 】 俄 伊尔库茨克州...


In [86]:
#提取特征
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['text'])
list_tokenized_train = tokenizer.texts_to_sequences(train_data['text'])
x = pad_sequences(list_tokenized_train, maxlen=100)
y = train_data['label']

In [87]:
#划分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [88]:
#模型搭建
from keras.models import Model, Sequential
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [89]:
#训练模型
model.fit(x_train, y_train, batch_size=50, epochs=5, validation_split=0.2)

Train on 3315 samples, validate on 829 samples
Epoch 1/5
3315/3315 [==============================] - 5s 2ms/step - loss: -0.4905 - acc: 0.5077 - val_loss: -2.0382 - val_acc: 0.5090
Epoch 2/5
3315/3315 [==============================] - 4s 1ms/step - loss: -6.0440 - acc: 0.5086 - val_loss: -11.1382 - val_acc: 0.5090
Epoch 3/5
3315/3315 [==============================] - 5s 1ms/step - loss: -12.3703 - acc: 0.5086 - val_loss: -11.4905 - val_acc: 0.5090
Epoch 4/5
3315/3315 [==============================] - 4s 1ms/step - loss: -12.4227 - acc: 0.5086 - val_loss: -11.5020 - val_acc: 0.5090
Epoch 5/5
3315/3315 [==============================] - 4s 1ms/step - loss: -12.4332 - acc: 0.5086 - val_loss: -11.5079 - val_acc: 0.5090


In [91]:
#评估
accuracy = model.evaluate(x_test, y_test, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))

1777/1777 [==============================] - 0s 54us/step
test accuracy:0.49577939788746445
